In [1]:
import vllm

In [2]:
!ls /usr/local/python/3.12.1/lib/python3.12/site-packages/vllm/

_C.abi3.so		       device_allocator    py.typed
__init__.py		       distributed	   ray
__pycache__		       engine		   reasoning
_aiter_ops.py		       entrypoints	   sampling_params.py
_bc_linter.py		       env_override.py	   scalar_type.py
_custom_ops.py		       envs.py		   scripts.py
_flashmla_C.abi3.so	       forward_context.py  sequence.py
_flashmla_extension_C.abi3.so  inputs		   tasks.py
_ipex_ops.py		       logger.py	   third_party
_moe_C.abi3.so		       logging_utils	   tokenizers
_version.py		       logits_process.py   tracing.py
assets			       logprobs.py	   transformers_utils
attention		       lora		   triton_utils
beam_search.py		       model_executor	   usage
benchmarks		       multimodal	   utils
collect_env.py		       outputs.py	   v1
compilation		       platforms	   version.py
config			       plugins		   vllm_flash_attn
connections.py		       pooling_params.py
cumem_allocator.abi3.so        profiler


In [3]:
!ls /usr/local/python/3.12.1/lib/python3.12/site-packages/vllm/v1/

__init__.py		 kv_cache_interface.py	serial_utils.py
__pycache__		 kv_offload		spec_decode
attention		 metrics		structured_output
core			 outputs.py		utils.py
cudagraph_dispatcher.py  pool			worker
engine			 request.py
executor		 sample


In [5]:
!ls /usr/local/python/3.12.1/lib/python3.12/site-packages/vllm/v1/worker

__init__.py			    kv_connector_model_runner_mixin.py
__pycache__			    lora_model_runner_mixin.py
block_table.py			    tpu_input_batch.py
cpu_model_runner.py		    tpu_model_runner.py
cpu_worker.py			    tpu_worker.py
dp_utils.py			    ubatch_utils.py
ec_connector_model_runner_mixin.py  ubatching.py
gpu				    utils.py
gpu_input_batch.py		    worker_base.py
gpu_model_runner.py		    xpu_model_runner.py
gpu_ubatch_wrapper.py		    xpu_worker.py
gpu_worker.py


In [4]:
from vllm.v1.worker.gpu_model_runner import GPUModelRunner
GPUModelRunner.execute_model??

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 12-07 03:27:16 [importing.py:44] Triton is installed but 0 active driver(s) found (expected 1). Disabling Triton to prevent runtime errors.
INFO 12-07 03:27:16 [importing.py:68] Triton not installed or not compatible; certain GPU-related functions will not be available.
WARNING 12-07 03:27:16 [interface.py:221] Failed to import from vllm._C: ImportError('libcuda.so.1: cannot open shared object file: No such file or directory')


Signature:
GPUModelRunner.execute_model(
    self,
    scheduler_output: 'SchedulerOutput',
    intermediate_tensors: vllm.sequence.IntermediateTensors | None = None,
) -> vllm.v1.outputs.ModelRunnerOutput | vllm.sequence.IntermediateTensors | None
Docstring: <no docstring>
Source:   
    @torch.inference_mode()
    def execute_model(
        self,
        scheduler_output: "SchedulerOutput",
        intermediate_tensors: IntermediateTensors | None = None,
    ) -> ModelRunnerOutput | IntermediateTensors | None:
        if self.execute_model_state is not None:
            raise RuntimeError(
                "State error: sample_tokens() must be called "
                "after execute_model() returns None."
            )

        # self._draft_token_ids is None when `input_fits_in_drafter=False`
        # and there is no draft tokens scheduled. so it need to update the
        # spec_decoding info in scheduler_output with async_scheduling.
        # use deepcopy to avoid the modificatio

In [6]:
GPUModelRunner._update_states??

Signature: GPUModelRunner._update_states(self, scheduler_output: 'SchedulerOutput') -> None
Source:   
    def _update_states(self, scheduler_output: "SchedulerOutput") -> None:
        """Update the cached states and the persistent batch with the scheduler
        output.

        The updated states are used by the `_prepare_inputs` function to create
        the input GPU tensors for the model.

        The SamplingMetadata is updated and copied to the GPU if there is a
        new/resumed/paused/finished request in the batch.
        """
        # Remove finished requests from the cached states.
        for req_id in scheduler_output.finished_req_ids:
            self.requests.pop(req_id, None)
            self.num_prompt_logprobs.pop(req_id, None)
        # Remove the finished requests from the persistent batch.
        # NOTE(woosuk): There could be an edge case where finished_req_ids and
        # scheduled_req_ids overlap. This happens when a request is aborted and
        # the